<a href="https://colab.research.google.com/github/amunoz88/RIIA-optimizacion-ganancia/blob/master/notebooks/revenue_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import tensorboard
import datetime
import tensorflow
from google.colab import files
from tensorflow import keras
import bisect
%load_ext tensorboard

#Descripción del problema#
Queremos determinar los mejores precios para objectos que queremos vender. Por ejemplo, cuando vendemos anuncios en internet, nos gustaría saber cuanto debemos cobrar por cada anuncio.

En algunos casos, tenemos un historial de cuanto está dispuesto a pagar una persona por algún objeto. Por ejemplo en eBay tenemos transacciones históricas, en AirBnb vemos cuanto paga la gente por departamentos, ...

##Formalización
Consideremos un conjunto de valores históricos $b_1, \ldots, b_m$. Esto es lo que historicamente usuarios han estado dispuestos a pagar. Por ejemplo, cuánto se ha rentado una casa en Airbnb, cuánto se ha pagado por objectos en eBay... Nuestra meta es seleccionar un precio $p$ que maximice nuestras ganancias históricas. Notemos que si $b_i < p$, no obtenemos ninguna ganancia. Si $b_i > p$ entonces nuestra ganancia es $p$. Por lo tanto queremos maximizar:

$$ G(p) = \sum_{i=1}^m p \mathbf{1}\{b_i \geq p\}$$

### Variantes
El problem anterior puede tener las siguientes variantes
 * Subastas de segundo precio: En una subasta de segundo precio multiplles agentes dicen cuanto están dispuestos a pagar. El ganador es quien está dispuesto a pagar más y termina pagando el máximo entre la segunda ooferta más alta y un precio mínimo $p$. Si denotamos por $b^{(1)}$ la oferta más alta y por $b^{(2)}$ la segunda oferta más alta el problema se reduce a maximizar $$G(p) = \sum_{i=1}^m \max(b^{(2)}_i , p) \mathbf{1}\{b^{(1)}_i \geq p\}$$. Este tipo de subastas es muy común cuando se venden objetos en internet como anuncios y objetos en eBay.
 * Inclusión de covariantes (features). Podemos codificar la información sobre el objeto en venta y el comprador en un vector $x$. Nuestro precio puede ser una función de $x$. Por ejemplo, en airbnb el precio de un departamento depende del número de cuartos, de la temporada en que se visita la ciudad, ...Nuestros datos ahora corresponden a pares $(x_i, b_i)$, donde $b_i$ es el precio que alguien está dispuesto a pagar por el objeto descrito por $x_i$.  Nuestra meta es resolver $$\max_{p \in \mathcal{P}} G(p) = \sum_{i=1}^m p(x_i) \mathbf{1} \{ b_i \geq p(x_i)\},$$ donde la familia $\mathcal{P}$ contiene funciones que mapea atributos (features) a valores reales. Por ejemplo $\mathcal{P}$ puede ser un conjunto de funciones lineales o redes neuronales.
 


#Ejercicio 1
##Implementación de las funciones de ganancia. 
Implementa la función _ganancia con parametro p y que toma como argumento un np.array de ofertas. 


In [ ]:
#@title Implementación de la función de ganancia
def _ganancia(p, b):
  """ Función de ganancia.
  Args:
    p: Un scalar
    b: Un np.array de ofertas b_1, ... b_m
  Returns:
    La ganancia histórica si ponemos un precio p. 
  """

def ganancia(p, b):
  """ Función de ganancia con argumento de vector
  Args:
    p: Un np.array
    b: Un np.array de overtas b_1, ..., b_m
  Returns:
    Un np.array de ganancias por cada entrada en p.
  """
  return np.array([_ganancia(pp,b) for pp in p])


In [ ]:
#@title Plot de la función de ganancia para una sola oferta
b =  2.0#@param
bb = np.array(b)
p = np.linspace(0, 8, 100)
plt.plot(p, ganancia(p, bb))
plt.xlabel("p")
l=plt.ylabel("Ganancia")

In [ ]:
#@title Plot de la función de ganancia con 20 ofertas ~ U(0,1)
b = np.random.uniform(0, 1, 20)
p = np.linspace(0, 1, 100)
plt.plot(p, ganancia(p, b))

#Ejercicio 2
## Optimizar mis ganancias
Dado un conjunto de ofertas como obtener el precio que maximize mis ganancias?

In [ ]:
#@title Implementa una función que optimize la ganancia histórica
def optimizar_ganancias(b):
  """ Función para optimizar la ganacia histórica dadoo un vector de ofertas
     Args:
       b: Un vector de ofertas
     Returns:
       max: la máxima ganancia posible
       a_max: el precio que optimiza la ganancia

In [ ]:
#@title Cargar ofertas. Suban el archivo bids.txt del repositorio
def LoadBids():
  uploaded = files.upload()
  str_array = uploaded['bids.txt'].decode().split(",")
  return np.array([float(b) for b in str_array])
ofertas = LoadBids()

Saving bids.txt to bids.txt


In [ ]:
#@title Código de evaluación.
p = np.linspace(0, 15, 100)
plt.plot(p, ganancia(p, ofertas))
plt.ylabel("ganancia")
plt.xlabel("p")
init_time = time.time()*1000
m, am = optimizar_ganancias(ofertas)
running_time = time.time() * 1000 - init_time
plt.vlines(am, 0, m, color='red')
plt.hlines(m, 0, p[99], color='red')
print("Ganancia propuesta: %.4f. Solución en %.2f milisegundos" % (ganancia([am], ofertas), running_time))

#Ejercicio 3
Introducción de atributos. Información sobre cada objecto es codificada en un vector $x \in \mathbb{R}^d$. Queremos predecir cuánto está dispuesto una persona a pagar por ese objecto. 

Solución tradicional es regresión: Dados ejemplos $(x_1, b_1), \ldots, (x_n, b_n)$ encontrar un vector $w$ que minimize:

$$ \sum_{i=1}^n (p(x_i)  - b_i)^2$$.

¿Pódemos usar las predicciones $p(x_i)$ como precios?


Sube los archivos titulados ej3* del repositorio.  El siguient bloque crea los objetos:
* X: Una matriz que contiene un vector de atributos por cada renglón
* b: Un vector que contiene una oferta asociada a cada atributo
* Xtest: Una matriz de datos usada para evaluar un modelo
* btest: Vector de overtas usado para evaluar un modelo. 



In [ ]:

uploaded = files.upload()
X = np.load("ej3_features.npy").T
b = np.load("ej3_costs.npy")
Xtest= np.load("ej3_features_test.npy").T
btest = np.load("ej3_costs_test.npy")

In [ ]:
#@title Analiza el código que crea la structura de una red neuronal 
d = X.shape[1]
def vector_revenue(p, b):
  prices = np.maximum(p,0).reshape(-1)
  buy = p.reshape(-1) <=b.reshape(-1)
  return np.sum(prices * buy)

def model_revenue(model, X, b):
  return vector_revenue(model.predict(X), b)

def create_neural_network(regularizer=0.001):
  input = keras.Input((d, ))
  relu_layer = keras.layers.Dense(40, use_bias=True, activation=keras.activations.relu,
                                    kernel_regularizer=keras.regularizers.l2(regularizer))
  linear_layer = keras.layers.Dense(1, use_bias=True, kernel_regularizer = keras.regularizers.l2(regularizer))
  output = linear_layer(relu_layer(input))
  model = keras.Model(inputs = input,outputs=output)
  return model
# Implementa esta función de ganancia. La función es una versión suave de
# función de ganancia que implementaste in el ejercicio 1    
# def tf_ganancia_smooth(ytrue, ypred, gamma):

In [ ]:
#@title Entrena tu model con error cuadrático.
batch_size = 1000  #@param
epochs = 100 #@param
funcion_perdida = "cuadratica" #@param 

gamma = 2.0
square_loss = keras.losses.MeanSquaredError()
rev_loss = lambda ytrue, ypred: -tf_ganancia_smooth(ytrue, ypred, gamma)
model_loss = square_loss
if funcion_perdida == "ganancia": 
  model_loss = rev_loss

def train_model(batch_size, epochs, model_loss, regularizer = 0.001,verbose=False):
  model = create_neural_network(regularizer)
  model.compile(optimizer=keras.optimizers.Adam(0.1),
              loss= model_loss)
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model.fit(X, y=b,batch_size=batch_size, epochs=epochs,
            callbacks=tensorboard_callback,validation_split=0.1,shuffle=True,
            verbose=verbose)
  return model
  
model = train_model(batch_size, epochs, model_loss, verbose=True)

In [ ]:
#@title Evalua tu modelo en datos de entrenamiento y de evaluacion
print("Ganancia en datos de entrenamiento %.3f. En datos de evaluacion %.3f" 
      % (model_revenue(model, X, b), model_revenue(model,Xtest, btest)))

* ¿Cómo sabes si tu model es bueno o malo?
* ¿Cuál es una cota inferior para la ganancia? ¿Superior?
* Implementa tus cotas y averigua que tan bueno o malo es éste modelo.
* ¿Cómo lo podemos mejorar?
* Trata de implementar tf_ganancia_smooth y entrena tu modelo con funcion_perdida = "ganancia"

In [ ]:
#@title Opcional: visualiza el progreso de tu modelo en Tensorboard.
%tensorboard --logdir logs

### ¿Qué podemos hacer con predicciones que usan la pérdida cuadrática?
[1] Andrés Muñoz Medina, Sergei Vassilvitskii. Revenue optimization with approximate bid predictions. 
La solución es usar clusters. Clusters con menos varianza nos ayudan a obtener más ganancias.
#### Algoritmo:
* Entrenamos un model usando pérdida cuadrática
* Ponemos en k clusters las predicciones
* Para cada cluster encontramos el precio óptimo usando la funcion optimizar_ganancias. 

#### Predicción
* Predecimos la oferta usando el model
* Encontramos el cluster al que la predicción pertenece
* Usamos el precio óptimo calculado para ese cluster

El código para este algoritmo no es trivial así que lo implementamos abajo. Las funciones principales son:
* train_reserve_price_cluster_model
* evaluate_cluster_model




In [ ]:
#@title Código de clusters

def train_reserve_price_cluster_model(X, b, model,k):
  """ Entrena un modelo de clusters usando un modelo de keras.
  Args:
    X: Matriz de "features"
    b: Ofertas
    model: Model de keras que predice ofertas.
    k: Número de clusters deseados
  Returns:
    cutoffs: Los delimitadores de los clusters
    cluster_to_prices: Un diccionario que mapea un número de cluster
        a un precio de reserva.
  """
  clusters, cutoffs = cluster_data_with_model(X, model, k=k)
  cluster_to_bids = {}
  for i,c in enumerate(clusters):
    if c not in cluster_to_bids.keys():
      cluster_to_bids[c] = []
    cluster_to_bids[c].append(b[i])
  cluster_to_prices = {}
  cluster_to_prices[0] = 0.0
  n = len(cutoffs)
  cluster_to_prices[n] = cutoffs[n-1]
  for c,bids in cluster_to_bids.items():
    rev, p = optimizar_ganancias(bids)
    cluster_to_prices[c] = p
  return cutoffs, cluster_to_prices


def evaluate_cluster_model(X, b, model, cutoffs, cluster_to_prices):
  """ Evalua un modelo de clusters.

  Args:
    X = Matriz de "features"
    b = Ofertas
    model: Modelo de keras
    cutoffs: Los delimitadores para clusters generatods por
       train_reserve_price_cluster_model
    cluster_to_prices: Diccionario que mapea numero de un cluster
       a un precio de reserva
    Returns:
      La ganancia que se obtiene con este modelo.
  """
  prices = _get_prices(np.maximum(model.predict(X), 0).reshape(-1),
                      cutoffs, cluster_to_prices)
  return vector_revenue(prices, b)

def histogram_mean(v, weights=None):
  #print v
  #print weights
  if  weights.any() == None:
    weights = np.ones(len(v), float)
  total_weight = np.sum(weights)
  return np.sum(weights*v) /total_weight

def histogram_var(v, weights=None):
  if weights.any() == None:
    weights = np.ones(len(v), float)
  total_weight = np.sum(weights)
  mean = histogram_mean(v, weights)
  s2 = histogram_mean(v**2, weights)
  return s2 - mean**2

""" Minimize the clustered weighted variance to some power. That is
   \sum_{j=1}^k n_k \sigma_k^{2 * power}.
   Args:
     v = sorted list for which we want to minimize the weighted variance.
     k = number of clusters
     power = float that represents the value at which we want to raise the
             variance
     weights = Optional, it should have the same length as v. weights[i]
               represents the weight that should be given to v[i].
     Returns:
     The optimal clustering, cutoff values and the weighted variance
     of the cluster. The clustering is represented by the indices of the cutoffs.
     A new element x belongs to cluster i if cutoff[i-1] < x <= cutoff[i].
     Where cutoff[-1] = -Inf and cutoff[k] = Inf.
     Complexity O(len(v)**2)"""
def DP_weighted_var(v, k, power, weights=None):
  if weights == None:
    weights = np.ones(len(v))
  n = len(v)
  # Dynamic programming matrix
  DPmat = np.ones((n, k)) * 2**32
  # Matrix to recover the indices of the optimal clustering
  DPindex_mat = np.zeros((n, k), np.int)
  DPindex_mat[0, 0] = 0
  DPmat[0, 0] = 0.0
  for i in range(1, n):
    for j in range(min(i + 1, k)):
      if j == 0:
        # If only one cluster then the
        DPmat[i, j] = (histogram_var(v[0:i + 1], weights[0:i+1]) **power
                       * np.sum(weights[0:i+1]))
        DPindex_mat[i, 0] = 0
      else:
        opt_value = 2**32
        opt_index = 0
        mean = v[i]
        s2 = v[i]**2
        total_current_weight = weights[i]
        for s in range(i):
          reverse_index = i - s
          value = 0 
          if s == 0:
             value = DPmat[i-1, j-1]
          else :
            mean = ((total_current_weight * mean
                    + v[reverse_index] * weights[reverse_index])/
                    (total_current_weight + weights[reverse_index]))
            s2 = (s2 * total_current_weight  + v[reverse_index]**2 * weights[reverse_index])/(total_current_weight + weights[reverse_index])
            total_current_weight += weights[reverse_index]
            value = (s2 - mean**2) ** power * total_current_weight + DPmat[reverse_index - 1, j-1]
          index = reverse_index
          if value <= opt_value:
            opt_value = value
            opt_index = index
        DPmat[i, j] = opt_value
        DPindex_mat[i, j] = opt_index
  # Recover the solution from the DP matrix.
  indices = [0] * k
  current_index = n - 1
  for j in range(0, k)[::-1]:
    indices[j] = DPindex_mat[current_index, j]
    current_index = indices[j] - 1
  return (indices, [v[s] for s in indices], DPmat[n - 1, k - 1])

def _assign_to_cutoff(value, cutoffs):
  if value < cutoffs[0]:
    return 0
  if value > cutoffs[len(cutoffs) -1]:
    return len(cutoffs)
  return bisect.bisect_right(cutoffs, value)
def _assign_to_cutoffs(values, cutoffs):
  return [_assign_to_cutoff(value, cutoffs) for value in values]

def _train_cluster_model(predictions, k):
  sorted_pred = quantize_predictions(predictions, 1000)
  _, cutoffs, _ = DP_weighted_var(sorted_pred,k, 1.0/3.0)
  return  _assign_to_cutoffs(predictions, cutoffs), cutoffs

def quantize_predictions(predictions, num_quantiles):
  if num_quantiles > len(predictions):
    return np.sort(predictions)
  return np.quantile(predictions,
                    np.linspace(1.0/num_quantiles, 1.0, num_quantiles))
  
def cluster_data_with_model(X, model, cutoffs=None, k=None):
  predictions = np.maximum(model.predict(X),0).reshape(-1)
  if cutoffs != None:
    return _assign_to_cutoffs(predictions, cutoffs), cutoffs
  return _train_cluster_model(predictions, k)

def _get_prices(predictions, cutoffs, cluster_to_prices):
  clusters = _assign_to_cutoffs(predictions, cutoffs)
  prices = np.zeros_like(predictions)
  for i,c in enumerate(clusters):
    prices[i] = cluster_to_prices[c]
  return prices


  
